In [ ]:
# default_exp input_file

# Input file

> Convenience function to construct the input file required by py2gift.

In [ ]:
# export

import sys
import pathlib
import pprint
import inspect
from typing import Union, Optional, Callable, List

import py2gift.util

import yaml

In [ ]:
test_file = '_input_.yaml'

# Reading

Code to parse input files.

### `extract_class_settings`

In [ ]:
# export

def extract_class_settings(category_name: Union[str, list], class_name: str, settings: dict):
    
    category_found = False
    
    for cat in settings['categories']:
        
        if cat['name'] == category_name:
            
            category_found = True
            
            for cls in cat['classes']:
                
                if cls['name'] == class_name:
                    
                    return cls
    
    else:
        
        if category_found:
            
            print(f'cannot find the requested class, {class_name}')
            sys.exit(1)
        
        else:
            
            print(f'cannot find the requested category, {category_name}')
            sys.exit(1)

In [ ]:
%%writefile {test_file}

output file: third_midterm.yaml
pictures base directory: tc/midterm3
path to gift-wrapper: '~/gift-wrapper/wrap.py'

categories:

  - name: Entropy of the input given the output

    classes:

      - name: EntropyOfInputGivenOutput

        question base name: Entropy of input given the output
        
        init parameters:
            
            picture_file: DMCs/entropy_at_input_given_output.tex

        statement: |
          Consider...
          

        feedback: |
          One way of tackling the problem is...
          

        number of instances: 2


In [ ]:
with open(test_file) as yaml_data:

    settings = yaml.load(yaml_data, Loader=yaml.FullLoader)

extract_class_settings('Entropy of the input given the output', 'EntropyOfInputGivenOutput', settings)

# Managing the settings with an *ad-hoc* object

A class to manage (*in memory*) all the settings required to build the questions.

In [ ]:
# export

class Settings:
    
    def __init__(self, output_file: str = 'quiz.yaml', pictures_directory: str = 'quiz/pics', test_mode: bool = False) -> None:
        
        self.test_mode = test_mode
        
        self.store = {}
        
        self.store['output file'] = output_file
        self.store['pictures base directory'] = pictures_directory
        self.store['categories'] = None
        
        self._classes = set()
    
    def to_dict(self) -> dict:
        
        return self.store
    
    def __repr__(self) -> str:
        
        return pprint.pformat(self.store)
    
    @property
    def fake_module(self) -> 'ClassesContainer':
    
        # [0] is the caller of the method (`stack`)
        caller_globals = inspect.stack()[1].frame.f_globals
        
        class ClassesContainer:
            
            pass
        
        class_container = ClassesContainer()
        
        for c in self._classes:
            
#             assert c in globals(), f'class "{c}" was not defined'
            assert c in caller_globals, f'class "{c}" was not defined'
            
#             setattr(class_container, c, globals()[c])
            setattr(class_container, c, caller_globals[c])
        
        return class_container

        
    def add_category(self, category_name: str, base_category: Optional[str] = None) -> Union[str, List[str]]:
        """
        Adds a category if it doesn't exist.
        
        
        Parameters
        ----------
        category_name: str
            The name of the category to be added.
        base_category: str, optional
            Parent of the category
            
        Returns
        -------
        out: str or list of str
            The category *actually* added.
        
        """
        
        assert type(category_name) == str

        if self.test_mode:

            category_name = 'test'

        else:

            if base_category:

                category_name = [base_category, f'{base_category}/{category_name}']


        if self.store['categories'] is None:

            self.store['categories'] = []
        
        # only if the category doesn't exist...
        if self.locate(category_name=category_name) is None:

            # ...is it added
            self.store['categories'].append({'name': category_name, 'classes': None})

        return category_name
    
    def locate(self, category_name: Union[str, list], class_name: Optional[str] = None) -> dict:
        """
        Returns the requested category or class (inside a category) or `None` if it can't be found.
        
        Parameters
        ----------
        category_name: str
            The name of the category.
        class_name: str, optional
            The name of the class.
            
        Returns
        -------
        out: dict
            The dictionary for the category or class.
        
        """
        
        for cat in self.store['categories']:
            
            if cat['name'] == category_name:
                
                # if no particular class was requested...
                if class_name is None:
                    
                    return cat
                
                # if a particular class was requested...
                else:
                
                    # the found category is saved for use below
                    category_settings = cat
                
                    # this is the only way out of the loop without return
                    break
        
        else:
            
            return None
        
            
        # at this point the category has been found and `class_name` is not `Ǹone`
        
        # if the category doesn't have any class...
        if category_settings['classes'] is None:
            
            return None
        
        for cls in category_settings['classes']:

            if cls['name'] == class_name:

                return cls

        else:

            return None
        
    def add_or_update_class(
        self, category_name: Union[str, list], class_name: str, question_base_name: str,
        init_parameters: Optional[dict] = None, parameters: Optional[List[dict]] = None,
        n_instances: Optional[int] = None, time: Optional[int]=None) -> None:

        d = {'name': class_name, 'question base name': question_base_name}

        if init_parameters:

            d['init parameters'] = init_parameters
            
        assert (parameters is not None) ^ (n_instances is not None)

        if parameters:

            d['parameters'] = parameters

        else:

            d['number of instances'] = n_instances

        if time:

            d['time'] = time
        
        class_settings = self.locate(category_name, class_name)
        
        if class_settings is None:
            
            category_settings = self.locate(category_name)
            
            assert category_settings is not None, f'category "{category_name}" not found'
            
            if category_settings['classes'] is None:
                
                category_settings['classes'] = []
            
            category_settings['classes'].append(d)
        
        else:
            
            class_settings.update(d)
        
        self._classes.add(class_name)

The object right after initialization:

In [ ]:
settings = Settings()
settings

A *scalar* category is added (it doesn't have any class yet)

In [ ]:
settings.add_category('My category')
settings

If `test_mode` is set to `True` when instantiating the class,

In [ ]:
settings = Settings(test_mode=True)

then the passed category name is ignored and *test* is used

In [ ]:
category_name = settings.add_category('My category')
settings

In such case, it is important to retrieve the value returned by `add_category`, since that is the *actual* category name added.

In [ ]:
settings.add_or_update_class(
    category_name=category_name, class_name='MeanLinearCombinationCosineAndUniform', question_base_name='Mean of a random process',
    init_parameters=dict(
        uniform_low_range=[2, 8], uniform_high_range=[12, 30], mean_range=[-5, 5],
        variance_range=[1, 10], cosine_frequency_over_pi=[1, 2, 3], t_range=[0, 30]
    ),
    n_instances=2, time=4
)
settings

In [ ]:
settings = Settings()
settings.add_category('My category')
settings.add_or_update_class(category_name='My category',class_name='A', question_base_name='question for class #1', n_instances=2, time=4)
settings.add_or_update_class(category_name='My category',class_name='B', question_base_name='question for class #2', n_instances=2, time=10)
settings.add_category(category_name='My category #1',base_category='Parent')
settings.add_or_update_class(
    category_name=['Parent', 'Parent/My category #1'],class_name='C', question_base_name='question for class #2', n_instances=2, time=10)

In [ ]:
settings

In [ ]:
settings.locate('My category', 'B')

In [ ]:
settings.locate(category_name=['Parent', 'Parent/My category #1'])

If the classes do exist

In [ ]:
class A:
    
    name = 'A'

class B:
    
    name = 'B'

class C:
    
    name = 'C'

In [ ]:
classes_container = settings.fake_module
print(classes_container.A.name)
print(classes_container.B.name)

# Composing a dictionary with the *YAML* settings

In [ ]:
# export

def initialize(output_file: str, pictures_directory: str, ) -> dict:
    
    settings = {}
    
    settings['output file'] = output_file
    settings['pictures base directory'] = pictures_directory
    settings['categories'] = None
    
    return settings

In [ ]:
settings_dict = initialize(output_file='quiz.yaml', pictures_directory='quiz/pics')

In [ ]:
# export

def set_class_preamble(settings: dict, category_name: str, base_category: Optional[str] = None, test_mode: bool = False) -> Union[str, list]:
    
    if test_mode:

        category_name = 'test'

    else:

        if base_category:

            category_name = [base_category, f'{base_category}/{category_name}']
    
    
    if settings['categories'] is None:
        
        settings['categories'] = []
        
    settings['categories'].append({'name': category_name, 'classes': None})
    
    return category_name

In [ ]:
set_class_preamble(settings_dict, 'Test category')
settings_dict

In [ ]:
# export

def set_class_closing(settings: dict, n_instances: int, time: Optional[int] = None) -> None:
    
    if settings['categories'][-1]['classes'] is None:
        
        settings['categories'][-1]['classes'] = [{}]
    
    settings['categories'][-1]['classes'][-1]['number of instances'] = n_instances
    settings['categories'][-1]['classes'][-1]['time'] = time

In [ ]:
set_class_closing(settings_dict, 2, 15)
settings_dict

In [ ]:
# export

def set_class(
    settings: dict, class_name: str, question_base_name: str, init_parameters: Optional[dict] = None,
    parameters: Optional[List[dict]] = None, n_instances: Optional[int] = None, time: Optional[int]=None) -> None:
    
    assert (parameters is not None) ^ (n_instances is not None)

    d = {'name': class_name, 'question base name': question_base_name}
    
    if init_parameters:
        
        d['init parameters'] = init_parameters
    
    if parameters:
        
        d['parameters'] = parameters
    
    else:
        
        d['number of instances'] = n_instances
    
    if time:
        
        d['time'] = time
    
    settings['categories'][-1]['classes'] = [d]

In [ ]:
set_class(
    settings_dict, class_name='MeanLinearCombinationCosineAndUniform', question_base_name='Mean of a random process',
    init_parameters=dict(
        uniform_low_range=[2, 8], uniform_high_range=[12, 30], mean_range=[-5, 5],
        variance_range=[1, 10], cosine_frequency_over_pi=[1, 2, 3], t_range=[0, 30]
    ), n_instances=2, time=4)

In [ ]:
settings_dict

# Writing

Code to write input files.

## Header

A function to write the header of the file.
* `file`: name of the input file to be created

In [ ]:
# export

def write_header(file: Union[str, pathlib.Path], output_file: str, pictures_directory: str, ) -> None:
    
    settings = initialize(output_file, pictures_directory)
    
    py2gift.util.dict_to_yaml(settings, file)

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')

In [ ]:
%cat {test_file}

## Class

### `write_class_preamble`

A function to write the preamble for a class which includes the name of the category. Strictly speaking, it's not necessary to create a new category every time a class is added, but this is more general and a category can show up many times in the input file.
* `file`: name of the input file to be *appended to*
* `category_name`: name of the category in which the class will be encompassed
* `base_category`: a parent category for `category_name` (optional)
* `test_mode`: if `True`, the last two parameters are overriden and the category is simply called `test`

For convenience, the final `category_name` (notice that it might get tweaked because of the other parameters) is returned.

In [ ]:
# export

def write_class_preamble(file: Union[str, pathlib.Path], category_name: str, base_category: Optional[str] = None, test_mode: bool = False) -> Union[str, list]:
    
    
    settings = py2gift.util.yaml_to_dict(file)
    category_name = set_class_preamble(settings, category_name, base_category, test_mode)
    py2gift.util.dict_to_yaml(settings, file)
    
    return category_name

In [ ]:
write_class_preamble(test_file, 'Test category')

In [ ]:
%cat {test_file}

Parameter `base_category` allows to create hierarchical categories

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base')

In [ ]:
%cat {test_file}

In `test_mode`

In [ ]:
write_header(test_file, output_file='quiz.yaml', pictures_directory='quiz/pics')
write_class_preamble(test_file, 'Test category', base_category='base', test_mode=True)

In [ ]:
%cat {test_file}

### `write_class_closing`

A function to write the closing settings of a class.
* `file`: name of the input file to be *appended to*
* `n_instances`: number of questions of this class that will be created (the `setup` method of the class should be non-deterministic)
* `time`: time in minutes (an *integer*) estimated necessary to solve the question

In [ ]:
# export

def write_class_closing(file: Union[str, pathlib.Path], n_instances: int, time: Optional[int] = None) -> None:
    
    settings = py2gift.util.yaml_to_dict(file)
    category_name = set_class_closing(settings, n_instances, time)
    py2gift.util.dict_to_yaml(settings, file)

In [ ]:
write_class_closing(test_file, 2, 15)

In [ ]:
%cat {test_file}

In [ ]:
!rm {test_file}

# Categories

In [ ]:
# export

def function_to_make_hierarchical_category_name(base_category: str) -> Callable[[str], list]:
    
    def make_subcategory(category: str) -> list:
    
        return [base_category, f'{base_category}/{category}']
    
    return make_subcategory

In [ ]:
f = function_to_make_hierarchical_category_name('2020 exam')
f('Random')

In [ ]:
# hide

import nbdev.export
nbdev.export.notebook2script('40_input_file.ipynb')